In [ ]:
!pip install msgpack_numpy

In [ ]:
import msgpack
import msgpack_numpy
import numpy as np
from sys import argv
%tensorflow_version 2.x
import tensorflow as tf
%load_ext tensorboard
import os
import datetime

print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

#enable numpy in msgpack files
msgpack_numpy.patch()

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
featurevector_length= 10

group_pointclouds = 2
sequence_length = 50

files = [#"adult_EWI-25.msgpack",
        #"achtertuinheuvel-1.msgpack" ,
         #"EWI_2_avond-25.msgpack",
       #  "EWI_3-26.msgpack" ,
         #"EWI_solarpanel-29.msgpack",
        #"schoolpleinheuvel-1.msgpack",
     #("ewitest-18.msgpack",2.0,0),
    ("football_children.msgpack",1.80,-3.3),
    ("football_2-31.msgpack",1.80,-3.3),
    ("football_3-31.msgpack",1.80,-3.3),
    ("adults-31.msgpack",1.80,-3.3),
    ("fietsen-20.msgpack",2.0,-2.6),
    ("fietsen2-20.msgpack",2.0,-2.6),
    #("one_at_a_time-31.msgpack",1.80,-3.3),
    ("mixed-31.msgpack",1.80,-3.3),
]

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def get_featurevector(data, height, angle):
    """
     Data = [range, angle, doppler, elevation, snr]
    """
    #print(data)
    #points = np.sum((np.sum(data, axis=2) != 0), axis=1)
    points = data.shape[0]

    summed = np.sum(data, axis=0)
    averaged = np.mean(data,axis=0)
    #deviation = np.std(data, axis=1)

    x, y = pol2cart(data[:,0], data[:,1])
    # featurevecs = np.zeros((featurevector_length))

    # _, elevation = pol2cart(data[:,0], data[:,4]+(3.1415/180*angle))
    # elevation += height
    # featurevecs[0] = averaged[0]
    # featurevecs[1] = averaged[1]
    # featurevecs[2] = averaged[2]
    # featurevecs[3] = np.percentile(elevation, 95)
    # featurevecs[4] = averaged[3]
    # featurevecs[5] = np.percentile(elevation, 5)
    # featurevecs[6] = np.std(x)#deviation[0]
    # featurevecs[7] = np.std(y)#deviation[1]
    # #featurevecs[6] = 0#np.mean(elevation)#np.mean(data[:,3]) / ((1/(averaged[0]/1400) +130)) if averaged[0] > 6 else (np.mean(data[:,3])/360)
    # #featurevecs[7 ] = points
    # #Out: [num points, range, angle, doppler, snr tot, snr avg, angle stdev, doppler stdev, rangedev, snr stdev ]

    #COPIED FROM RFCLASSIFIER
    variance = np.var(data, axis=0)

    featurevecs = np.zeros((featurevector_length))

    x, y = pol2cart(data[:,0], data[:,1])

    _, elevation = pol2cart(data[:,0], data[:,4]+(3.1415/180*angle))
    elevation += height
    featurevecs[0] = averaged[0] #range
    featurevecs[1] = averaged[1] #angle
    featurevecs[2] = averaged[2] #doppler
    featurevecs[3] = np.percentile(elevation, 95)
    #featurevecs[10] = np.sum(data[:,3], ) #snr

    featurevecs[4] = np.std(x)#deviation[0]
    featurevecs[5] = np.std(y)#deviation[1]
    #featurevecs[7] = deviation[2]
    #featurevecs[8] = deviation[3]

    featurevecs[6:9] = variance[0:3]

    # featurevecs[10] = np.percentile(elevation, 95)
    # featurevecs[11] = np.percentile(elevation, 5)

    return featurevecs


def read_file(filename):
    """
    read a messagepack file and return individual messages
    :return: 
    """
    with open(filename, 'rb') as file:
        unpacker = msgpack.Unpacker(file, raw=False)
        for msg in unpacker:
            yield msg


def get_pointclouds(msg):
    """
    get pointcloud data from msg
    :param msg:
    :return:
    """
    return msg['pointclouds']


def get_dataset(fileinfo, id):
    feature_vectors = []
    labels = []
    ids = []
    filename = fileinfo[0]
    sensorheight = fileinfo[1]
    sensorangle = fileinfo[2]
    for msg in read_file(filename):
        msg_feature_vectors = []
        msg_labels = 0
        pointclouds = get_pointclouds(msg)
        if(len(pointclouds)  > 100):
            class_id = msg['class_id']
            if(class_id >=0):
                i = 0
                sequence =[]
                while i < len(pointclouds):
                    pointcloud = pointclouds[i]
                    i += 1
                    if (pointcloud.shape[0] > 1):
                        for j in range(min(group_pointclouds - 1, len(pointclouds)-i)):
                            pointcloud2 = pointclouds[i]
                            if (pointcloud2.shape[0] > 1):
                                pointcloud = np.append(pointcloud, pointcloud2, axis=0)
                            i += 1
                        fv = get_featurevector(pointcloud, sensorheight, sensorangle)
                        sequence.append(fv)

                if(len(sequence) >= sequence_length):
                    for j in range(0,len(sequence)-sequence_length-1):
                        feature_vectors.append(np.array(sequence[j : j+sequence_length]))
                        labels.append(msg['class_id'] if msg['class_id'] >= 0 else 3)
                        ids.append(id*100000+msg['uid'])

    # labels: [adult, bike, child, clutter]

    labels = np.array(labels)
    features = np.array(feature_vectors)
    ids = np.array(ids)
    return labels, features, ids

features = []
labels = []
ids = []
for j in range(0, len(files), 1):
    a, b, c = get_dataset(files[j],j)
    features.append(b)
    labels.append(a)
    ids.append(c)
    print(b.shape)

#print(features)
labels = np.concatenate(labels, axis=0)
features = np.concatenate(features, axis=0)
ids = np.concatenate(ids, axis=0)

#b = tf.keras.utils.normalize(b, axis=0, order=2)
print(features.shape, labels.shape)
print(np.unique(labels,return_counts=True))

(1735, 50, 10)
(47, 50, 10)
(1532, 50, 10)
(2105, 50, 10)
(1583, 50, 10)
(670, 50, 10)
(2811, 50, 10)
(10483, 50, 10) (10483,)
(array([0, 1, 2]), array([4099, 2253, 4131]))


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(16, input_shape=(sequence_length,featurevector_length)),
    #tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3, activation='softmax'),    
])


dataset = tf.data.Dataset.from_tensor_slices((features,labels))


BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 50000

dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)
test_dataset = dataset.take(1024).batch(BATCH_SIZE)
train_dataset = dataset.skip(1024).batch(BATCH_SIZE)

    
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)    
model.compile(  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
                loss=loss,
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


val_files = [
    ("one_at_a_time-31.msgpack",1.80,-3.3),
    ("fietsen4-20.msgpack",2.0,-2.6),
    #("mixed-31.msgpack",1.80,-3.3),
 ]
val_features = []
val_labels = []
for j in range(0, len(val_files), 1):
    a, b, c = get_dataset(val_files[j],j)
    val_features.append(b)
    val_labels.append(a)
    print(b.shape)

val_labels = np.concatenate(val_labels, axis=0)
val_features = np.concatenate(val_features, axis=0)

validation = tf.data.Dataset.from_tensor_slices((val_features,val_labels)).batch(BATCH_SIZE)

# #model.load_weights('easy_checkpoint')
model.fit(train_dataset, epochs=15, validation_data=validation, callbacks=[tensorboard_callback])
#model.save_weights('easy_checkpoint')
# res = model(b)
# #print(a.shape, np.argmax(res.numpy(),axis=1).shape)
# print(tf.math.confusion_matrix(np.argmax(a,axis=1), np.argmax(res.numpy(),axis=1)))

(3360, 50, 10)
(431, 50, 10)
Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

148/148 [==============================] - 3s 19ms/step - loss: 1.0307 - sparse_categorical_accuracy: 0.5796 - val_loss: 1.0047 - val_sparse_categorical_accuracy: 0.6737
Epoch 2/15
148/148 [==============================] - 2s 15ms/step - loss: 0.9140 - sparse_categorical_accuracy: 0.7191 - val_loss: 0.8492 - val_sparse_categorical_accuracy: 0.8430
Epoch 3/15
148/148 [==============================] - 2s 15ms/step - loss: 0.8017 - sparse_categorical_accuracy: 0.82

In [ ]:

predicted = np.argmax(model(val_features), axis=1)
print(np.sum(predicted == val_labels) / len(val_labels))
tf.math.confusion_matrix(val_labels, predicted,3)

0.9105776839883936


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1763,    0,   50],
       [  32,  398,    1],
       [ 256,    0, 1291]], dtype=int32)>

In [ ]:
%tensorboard --logdir logs

<IPython.core.display.Javascript object>

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch multiple                  40        
_________________________________________________________________
dropout_8 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  176       
_________________________________________________________________
dropout_9 (Dropout)          multiple                  0         
_________________________________________________________________
lstm_1 (LSTM)                multiple                  2112      
_________________________________________________________________
dense_8 (Dense)              multiple                  51        
Total params: 2,379
Trainable params: 2,359
Non-trainable params: 20
___________________________________________________

In [ ]:

val_files = [
    ("one_at_a_time-31.msgpack",1.80,-3.3),
    ("fietsen4-20.msgpack",2.0,-2.6),
 #   ("mixed-31.msgpack",1.80,-3.3),
 ]

sequence_length=50

def get_dataset(fileinfo, id):
    feature_vectors = []
    labels = []
    ids = []
    filename = fileinfo[0]
    sensorheight = fileinfo[1]
    sensorangle = fileinfo[2]
    for msg in read_file(filename):
        msg_feature_vectors = []
        msg_labels = 0
        pointclouds = get_pointclouds(msg)
        if(len(pointclouds)  > 100):
            class_id = msg['class_id']
            if(class_id >=0):
                i = 0
                sequence =[]
                while i < len(pointclouds):
                    pointcloud = pointclouds[i]
                    i += 1
                    if (pointcloud.shape[0] > 1):
                        for j in range(min(group_pointclouds - 1, len(pointclouds)-i)):
                            pointcloud2 = pointclouds[i]
                            if (pointcloud2.shape[0] > 1):
                                pointcloud = np.append(pointcloud, pointcloud2, axis=0)
                            i += 1
                        fv = get_featurevector(pointcloud, sensorheight, sensorangle)
                        sequence.append(fv)

                if(len(sequence) >= sequence_length):
                    for j in range(0,len(sequence)-sequence_length-1):
                        feature_vectors.append(np.array(sequence[j : j+sequence_length]))
                        labels.append(msg['class_id'] if msg['class_id'] >= 0 else 3)
                        ids.append(id*100000+msg['uid'])

    # labels: [adult, bike, child, clutter]

    labels = np.array(labels)
    features = np.array(feature_vectors)
    ids = np.array(ids)
    return labels, features, ids


val_features = []
val_labels = []
for j in range(0, len(val_files), 1):
    a, b, c = get_dataset(val_files[j],j)
    val_features.append(b)
    val_labels.append(a)
    print(b.shape)

val_labels = np.concatenate(val_labels, axis=0)
val_features = np.concatenate(val_features, axis=0)

(3360, 50, 10)
(431, 50, 10)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  40        
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  704       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
gru (GRU)                    multiple                  24960     
_________________________________________________________________
dense_1 (Dense)              multiple                  195       
Total params: 25,899
Trainable params: 25,879
Non-trainable params: 20
___________________________________________________

In [ ]:
32+288+8320+99

8739